In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install flair

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from bs4 import BeautifulSoup
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from transformers import BertTokenizer, BertModel, DistilBertConfig, TFDistilBertModel, DistilBertModel, DistilBertTokenizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
logging.getLogger("pytorch_pretrained_bert.tokenization").setLevel(logging.ERROR)
from flair.data import Sentence
from flair.embeddings import BertEmbeddings
import pickle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install -U sentence-transformers

In [ ]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

### Covid dataset

In [ ]:
# Load dataset for clothing reviews
covid_train = pd.read_csv("/content/drive/MyDrive/Comp550/CovidNlp/Corona_NLP_train.csv", encoding="latin1")
covid_train = covid_train.dropna()
covid_test = pd.read_csv("/content/drive/MyDrive/Comp550/CovidNlp/Corona_NLP_test.csv")
covid_test = covid_test.dropna()

covid_trian_inds = list(range(0, covid_train.shape[0]))

# cat both datasets for preprocessing
frames = [covid_train, covid_test]
covid = pd.concat(frames)
print(covid_train.head())
print(covid_test.shape)
print(type(covid))

   UserName  ...  Sentiment
0      3799  ...    Neutral
1      3800  ...   Positive
2      3801  ...   Positive
5      3804  ...   Positive
6      3805  ...   Positive

[5 rows x 6 columns]
(2964, 6)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
covid.head()

In [ ]:
# preprocess training and testing set

covid = covid[['OriginalTweet', 'Sentiment']]
covid.columns = ['tweet', 'sentiment']
sentiment_dict = {'negative': 0, 'extremely negative':0, 'neutral':1, 'positive':2, 'extremely positive':2}
error_line = []

def encode_sentiment(x):
    sent_encoded = sentiment_dict[x.lower()]
    return sent_encoded

covid['sentiment_encoded'] = covid['sentiment'].apply(lambda x: encode_sentiment(x))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
covid.head()

,tweet,sentiment,sentiment_encoded
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,1
1,advice Talk to your neighbours family to excha...,Positive,2
2,Coronavirus Australia: Woolworths to give elde...,Positive,2
5,As news of the regionÂs first confirmed COVID...,Positive,2
6,Cashier at grocery store was sharing his insig...,Positive,2


In [ ]:
print(len(covid.sentiment_encoded))
print(covid.shape[0])

35531
35531


#### Tokenize each sentence

In [ ]:
# read in data
def clear_data(df):
    stopword = set(stopwords.words('english'))

    # tokenize and post process, remove unnecessary words appearing in tweets(like urls and usernames)
    tweet_words = []
    str_lis = []
    for t in df['tweet']:
        t = t.lower()
        tokens = tokenize(t)
        tmp = []
        for token in tokens:
            # if token in stopword:
            #     continue
            if not wordnet.synsets(token):
                continue
            # repalce usernames with @USER
            user = re.sub(r'@[\w\W]+', '@USER', token)
            # replace with URL
            url = re.sub(r'https[\w\W]+','URL',user)
            if url in stopword:
                continue
            tmp.append(url)
        tweet_words.append(tmp)
        st = ' '.join([str(item) for item in tmp])
        str_lis.append(st)
    df['tweet_words'] = tweet_words
    df['tweets'] = str_lis

    # remove duplicates
    df.drop_duplicates(subset=['tweets'], inplace=True)

    df.drop(columns='tweet_words', inplace=True)
    print(len(df))
    df.to_csv("covid.csv", index=False)
    return df

def tokenize(text):
    words = []
    # print(text.split())
    for token in text.split():
        # find wouldn't kind word
        words.extend(re.findall(r"\w+-\w+|https.+|\.+|\d+[\.,]\d+|[@#]\w+|[+-]\d+|(?:(?!n[’'])\w)+|\w?[’']\w+|[^\s\w]", token))
        # words.extend(re.findall(r"(?:(?!.')\w)+|\w?'\w+|[^\s\w]", token))
    return words

In [ ]:
covid = clear_data(covid)

35000


In [ ]:
covid.head()

,tweet,sentiment,sentiment_encoded,tweets
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,1,
1,advice Talk to your neighbours family to excha...,Positive,2,advice talk neighbours family exchange phone n...
2,Coronavirus Australia: Woolworths to give elde...,Positive,2,australia woolworths give elderly disabled ded...
5,As news of the regionÂs first confirmed COVID...,Positive,2,news first confirmed case came sullivan county...
6,Cashier at grocery store was sharing his insig...,Positive,2,cashier grocery store sharing insights prove c...


#### Encode and truncate sentence

In [ ]:
# tokenize the tweets into list
def encode_tweets(x):
    tweet_list = x.split()
    return tweet_list

covid['tweets_tokenized'] = covid['tweets'].apply(lambda x: encode_tweets(x))
covid.reset_index(drop=True, inplace=True)
# drop rows wth empty tokenized tweet
for i, row in covid.iterrows():
    if len(row.tweets_tokenized) < 2:
        covid.drop(i, inplace = True)

In [ ]:
covid.head(30)

,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized
1,advice Talk to your neighbours family to excha...,Positive,2,advice talk neighbours family exchange phone n...,"[advice, talk, neighbours, family, exchange, p..."
2,Coronavirus Australia: Woolworths to give elde...,Positive,2,australia woolworths give elderly disabled ded...,"[australia, woolworths, give, elderly, disable..."
3,As news of the regionÂs first confirmed COVID...,Positive,2,news first confirmed case came sullivan county...,"[news, first, confirmed, case, came, sullivan,..."
4,Cashier at grocery store was sharing his insig...,Positive,2,cashier grocery store sharing insights prove c...,"[cashier, grocery, store, sharing, insights, p..."
5,Was at the supermarket today. Didn't buy toile...,Neutral,1,supermarket today buy toilet paper,"[supermarket, today, buy, toilet, paper]"
6,Due to COVID-19 our retail store and classroom...,Positive,2,due retail store classroom atlanta open walk-i...,"[due, retail, store, classroom, atlanta, open,..."
7,"For corona prevention,we should stop to buy th...",Negative,0,corona prevention stop buy things cash use onl...,"[corona, prevention, stop, buy, things, cash, ..."
8,All month there hasn't been crowding in the su...,Neutral,1,month crowding supermarkets restaurants howeve...,"[month, crowding, supermarkets, restaurants, h..."
9,"Due to the Covid-19 situation, we have increas...",Extremely Positive,2,due situation increased demand food products w...,"[due, situation, increased, demand, food, prod..."
10,#horningsea is a caring community. LetÂs ALL ...,Extremely Positive,2,caring community look less capable village ens...,"[caring, community, look, less, capable, villa..."


In [ ]:
# sanity check for length of tweet
len_list = [len(x) for x in covid.tweets_tokenized]
max(len_list)

39

In [ ]:
# get unique words in the corpus
all_words = []
for x in covid['tweets_tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in covid.tweets_tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in covid['tweets_tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in covid.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

covid['encoded_old'] = covid.tweets_tokenized.apply(lambda x: encode(x))


# get sequence average length
total_len = 0
for x in covid.encoded_old:
    total_len += len(x)

ave_len = math.floor(total_len/covid.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

def fix_tweet(x):
    size = min(len(x), ave_len)
    new_encoded = x[:size]
    len_before_pad = len(new_encoded)
    if size < ave_len:
        for j in range(0, ave_len - len(x)):
            new_encoded.append(0)
    return new_encoded

covid['encoded'] = covid.encoded_old.apply(lambda x: fix_tweet(x))
covid['tweet_length'] = covid.encoded_old.apply(lambda x: min(len(x), ave_len))

In [ ]:
max(covid.tweet_length)

print(word2index['exchange'])
covid.head()

7948


,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized,encoded_old,encoded,tweet_length
1,advice Talk to your neighbours family to excha...,Positive,2,advice talk neighbours family exchange phone n...,"[advice, talk, neighbours, family, exchange, p...","[16962, 6636, 9075, 11304, 7948, 1137, 760, 14...","[16962, 6636, 9075, 11304, 7948, 1137, 760, 14...",14
2,Coronavirus Australia: Woolworths to give elde...,Positive,2,australia woolworths give elderly disabled ded...,"[australia, woolworths, give, elderly, disable...","[10935, 1494, 9670, 18475, 8292, 6824, 10385, ...","[10935, 1494, 9670, 18475, 8292, 6824, 10385, ...",9
3,As news of the regionÂs first confirmed COVID...,Positive,2,news first confirmed case came sullivan county...,"[news, first, confirmed, case, came, sullivan,...","[19218, 8717, 9975, 3058, 10454, 17354, 7551, ...","[19218, 8717, 9975, 3058, 10454, 17354, 7551, ...",14
4,Cashier at grocery store was sharing his insig...,Positive,2,cashier grocery store sharing insights prove c...,"[cashier, grocery, store, sharing, insights, p...","[13231, 8136, 15868, 14551, 2866, 217, 446, 56...","[13231, 8136, 15868, 14551, 2866, 217, 446, 56...",12
5,Was at the supermarket today. Didn't buy toile...,Neutral,1,supermarket today buy toilet paper,"[supermarket, today, buy, toilet, paper]","[15331, 16132, 10391, 1114, 2639]","[15331, 16132, 10391, 1114, 2639, 0, 0, 0, 0, ...",5


In [ ]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in covid.encoded]
print(max(lengths))
print(min(lengths))
print(set(covid.sentiment_encoded))
tweet_length = covid.tweet_length
covid.head()
print(len(word_set))

510109
14
14
14
{0, 1, 2}
20347


In [ ]:
# Train test split from skearln
data_size = len(covid['encoded'])
assert data_size == len(covid['sentiment_encoded']) 
X, y = list(zip(list(covid['encoded']),(list(covid['tweet_length'])))), list(covid['sentiment_encoded'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [ ]:
# Sanity check for length match
print(X_train[2])
print(y_train[2])

([7601, 19369, 18978, 17628, 12674, 9015, 12579, 19450, 8578, 10184, 10385, 5597, 1241, 2474], 14)
2


## Dataset and Dataloader

In [ ]:
class Dataset_word2vec(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [ ]:
trainSet_w2v = Dataset_word2vec(X_train, y_train)
testSet_w2v = Dataset_word2vec(X_test, y_test)

In [ ]:
print(trainSet_w2v[2])

(([7601, 19369, 18978, 17628, 12674, 9015, 12579, 19450, 8578, 10184, 10385, 5597, 1241, 2474], 14), 2)


In [ ]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [ ]:
trainLoader_w2v = DataLoader(dataset = trainSet_w2v, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_w2v = DataLoader(dataset = testSet_w2v, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)

In [ ]:
it = iter(trainLoader_w2v)
x, y, seq_len = it.next()

In [ ]:
x

tensor([[ 4029, 17495,  1230, 19511, 20077,  7977,  8136, 15868, 19454,  1403,
         11592, 18727,  3274,  5072],
        [ 5283, 14470, 12612,  4529,  6323,  8136, 15868,  7883, 14518, 16220,
          4790,  7412, 14518,  9957],
        [ 3639,  6201, 14484,  1247, 20028, 13340,  3471,  8332,  1078, 15540,
          8332,  6539,  6588, 18485],
        [ 2348,  5283,  3478, 11752,  1977, 19297,  4675,  8136, 15868,  1927,
          4675, 17509, 11578,  5232],
        [11114, 19577, 10984, 17604,  1591, 16132, 17763,   216,  6824, 16470,
           169,  3043, 10437, 14049],
        [ 9914,  6118,  7326, 12712, 12246,  5694, 14049,  9450, 17449,     0,
             0,     0,     0,     0],
        [ 1272, 14700,  6346,  6192,  9708,  7075,  9190,     0,     0,     0,
             0,     0,     0,     0],
        [15331,  1409,  5238, 15868, 19577,  1158, 15465, 12579, 11004,  7547,
         10356,  9969, 15414,  7601],
        [13835, 17035,   613, 19556, 14689,  7448,  3491, 12256,

In [ ]:
y

tensor([0, 0, 0, 2, 2, 1, 0, 2, 0, 1, 2, 2, 2, 1, 2, 2])

In [ ]:
seq_len

[14, 14, 14, 14, 14, 9, 7, 14, 14, 5, 14, 14, 5, 14, 14, 14]

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embedding(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

## Embedding

In [ ]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    y_total = []
    y_pred_total = []
    for x, y, seq_len in valid_dl:
        x = x.long().to(device)
        y = y.long().to(device)
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        y_total.extend(y.tolist())
        y_pred_total.extend(pred.tolist())
        
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        # print(len(y_total))
        # print(len(y_pred_total))

      # calculate precision, recall, and f1
    print(y_total)
    print(y_pred_total)
    f1 = (f1_score(y_total, y_pred_total, average='macro'))
    precision = (precision_score(y_total, y_pred_total, average='macro'))
    recall =(recall_score(y_total, y_pred_total, average='macro'))
    return sum_loss/total, correct/total, precision, recall, f1

In [ ]:
def train_model(model, fname, epochs = 50, lr = 0.0001):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    PATH = '/content/drive/MyDrive/McGill/Comp550/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in tqdm(range(epochs)):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in trainLoader_w2v:
            x = x.long().to(device)
            y = y.long().to(device)
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]

        val_loss, val_acc, precision, recall, f1 = validation_metrics(model, testLoader_w2v)
        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f, precision %.3f, recall %.3f, F1 %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc, precision, recall, f1))
        if val_acc > best_val_acc and i>=2:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}'
            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [ ]:
vanilla_LSTM = LSTM_fixed_len(len(all_words), 50, 50)

In [ ]:
train_model(vanilla_LSTM,"vanilla_lstm")

  0%|          | 0/50 [00:00<?, ?it/s]

At epoch 0
current in vlaidaiton


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  2%|▏         | 1/50 [00:12<09:58, 12.22s/it]

[1, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 1, 0, 1, 2, 2, 0, 2, 1, 2, 2, 0, 2, 2, 0, 1, 1, 1, 2, 0, 0, 1, 0, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 1, 2, 2, 1, 1, 0, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 1, 1, 0, 0, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 2, 1, 2, 2, 0, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 1, 0, 1, 1, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 1, 0, 0, 1, 2, 1, 0, 0, 0, 2, 2, 0, 2, 1, 2, 0, 1, 2, 0, 0, 1, 2, 0, 2, 2, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 2, 1, 1, 2, 2, 0, 1, 2, 2, 1, 2, 1, 0, 1, 1, 2, 2, 0, 2, 2, 0, 0, 0, 2, 1, 0, 2, 0, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 1, 1, 0, 0, 2, 0, 0, 2, 2, 1, 1, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 1, 2, 0, 2, 1, 1, 0, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 1, 2, 0, 2, 2, 0, 2, 1, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 1, 0, 1, 0, 1, 2, 1, 0, 

  4%|▍         | 2/50 [00:24<09:47, 12.24s/it]

...

[1, 2, 0, 2, 1, 0, 0, 2, 0, 0, 2, 1, 2, 1, 2, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 1, 0, 2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 1, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 1, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 1, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 1, 0, 2, 1, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 1, 2, 0, 2, 0, 1, 2, 1, 2, 1, 1, 2, 0, 2, 1, 0, 2, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 2, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 2, 0, 2, 2, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 1, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 1, 0, 1, 1, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 1, 0, 1, 0, 2, 2, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 0, 2, 2, 0, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 1, 2, 0, 2, 2, 0, 1, 2, 1, 2, 0, 1, 2, 0, 0, 2, 1, 2, 0, 1, 0, 2, 0, 

 98%|█████████▊| 49/50 [10:00<00:12, 12.25s/it]

[1, 0, 0, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 1, 2, 2, 0, 1, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 1, 1, 2, 2, 0, 1, 2, 1, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2, 0, 0, 1, 2, 1, 0, 2, 0, 2, 0, 1, 2, 2, 1, 0, 2, 2, 0, 2, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 2, 2, 2, 1, 2, 0, 0, 2, 0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1, 2, 0, 2, 0, 2, 0, 2, 2, 1, 2, 0, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 1, 2, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 2, 2, 0, 1, 1, 0, 1, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2, 0, 0, 0, 2, 1, 1, 1, 1, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 0, 1, 0, 2, 

100%|██████████| 50/50 [10:12<00:00, 12.26s/it]

[2, 2, 2, 1, 0, 2, 0, 2, 1, 0, 1, 0, 2, 2, 0, 2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 1, 2, 1, 1, 2, 2, 0, 2, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 2, 0, 2, 2, 2, 0, 1, 2, 0, 2, 2, 0, 0, 2, 1, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 1, 2, 1, 1, 2, 1, 2, 0, 0, 2, 2, 2, 1, 1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 1, 2, 2, 2, 0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 1, 1, 1, 0, 1, 0, 2, 2, 0, 2, 1, 1, 2, 2, 0, 0, 0, 2, 2, 1, 0, 1, 2, 0, 2, 2, 2, 0, 2, 1, 1, 1, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 1, 1, 0, 2, 0, 2, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 0, 0, 1, 0, 1, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 1, 0, 